In [1]:
import os
%load_ext autoreload
%autoreload 2

In [12]:
import pandas as pd
import requests
import argparse
import typing
import json
from urllib.parse import quote_plus
from src.utils import df_na_vals, np_min_chunk, time_utc_now
from src.fetch import _apiurl
from src.preprocess import preprocess, split_raw
from src.tables_old import get_machines, get_offers
import os

In [13]:
time_utc_now()

Timestamp('2024-03-31 04:58:43+0000', tz='UTC')

In [17]:
from src import const
import logging

query = {"external": {"eq": False}, "disable_bundling": {"eq": True}, "type": "on-demand"}
url = _apiurl(const.VAST_API_BASEURL, "/bundles", {"q": query})

# url = const.VAST_EXPORTER_BASEURL + '/offers'
headers = {
  'Accept': 'application/json',
  'Authorization': 'Bearer YOUR_API_KEY'
}

r = requests.pop(url, timeout=20, )
r.raise_for_status()

r_dict = r.json()

ts = int(pd.to_datetime(r_dict.pop('timestamp', time_utc_now())).timestamp())
raw = pd.DataFrame(r.json()["offers"])
raw['timestamp'] = ts
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 77 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    64 non-null     int64  
 1   ask_contract_id       64 non-null     int64  
 2   bundle_id             64 non-null     int64  
 3   bundled_results       25 non-null     float64
 4   bw_nvlink             64 non-null     float64
 5   compute_cap           64 non-null     int64  
 6   cpu_arch              64 non-null     object 
 7   cpu_cores             63 non-null     float64
 8   cpu_cores_effective   64 non-null     float64
 9   cpu_ghz               63 non-null     float64
 10  cpu_name              61 non-null     object 
 11  cpu_ram               64 non-null     int64  
 12  credit_discount_max   64 non-null     float64
 13  cuda_max_good         64 non-null     float64
 14  direct_port_count     64 non-null     int64  
 15  disk_bw               64 

In [47]:
endpoint = 'bundles'

url = "https://console.vast.ai/api/v0/" + endpoint

payload = {
  # "verified": { "eq": True },
  "external": { "eq": False },
  "disable_bundling": { "eq": True },
  # "rentable": { "eq": True },
  # "compute_cap": { "gt": "600" },
  # "disk_space": { "gt": "10000" },
  # "order": [["score", "desc"]],
  "type": "on-demand"
}
headers = {
  'Accept': 'application/json',
  # 'Authorization': 'Bearer '
}

r = requests.pop(url, headers=headers, json=payload)
# r = requests.pop(url, timeout=20)
r.raise_for_status()

len(r.json()['offers'])

# ts = int(pd.to_datetime(r_dict.pop('timestamp', time_utc_now())).timestamp())
# raw = pd.DataFrame(r.json()["instances"])
# raw['timestamp'] = ts
# raw.info()

64

In [26]:
r.json()

{'machines': [{'clients': [{'id': 10315256,
     'last_update': 1711759515.6715374,
     'last_proc': 1711737093.759245,
     'type': 'contract',
     'label': 'Master24 (triple) 007',
     'client_id': 54033,
     'host_id': 33132,
     'created_at': 1710561081.4570918,
     'deleted_at': None,
     'start_date': 1711706466.8201187,
     'end_date': 1715810418.0,
     'client_run_time': 100000000.0,
     'client_stop_time': 1.0,
     'client_unload_time': 1.0,
     'host_run_time': 1.0,
     'host_stop_time': 2592000.0,
     'host_unload_time': 2592000.0,
     'next_trans_date': 2411737093.759245,
     'max_spend': None,
     'created_from': 10103633,
     'bundle_id': None,
     'last_billup': None,
     'is_system': 0,
     'earn_sec': None,
     'earn_min': None,
     'earn_hour': None,
     'earn_day': None,
     'loss_sec': None,
     'loss_min': None,
     'loss_hour': None,
     'loss_day': None,
     'min_bid_price': 0.4,
     'dlperf': 71.074188,
     'credit_balance': None,


In [18]:
url

'https://console.vast.ai/api/v0/bundles?q=%7B%22external%22%3A+%7B%22eq%22%3A+false%7D%2C+%22disable_bundling%22%3A+%7B%22eq%22%3A+true%7D%2C+%22type%22%3A+%22on-demand%22%7D'

In [15]:
raw

,id,ask_contract_id,bundle_id,bundled_results,bw_nvlink,compute_cap,cpu_arch,cpu_cores,cpu_cores_effective,cpu_ghz,...,verification,vericode,vram_costperhour,webpage,rn,reliability2,discount_rate,discounted_hourly,discounted_dph_total,timestamp
0,10335846,10335846,1598311125,NaN,0.0,890,amd64,256,21.333333,2.250,...,verified,1,0.008337,None,1,0.988346,0.0,0.0,0.201667,1711861114
1,9581762,9581762,1330075774,3617.0,0.0,890,amd64,56,56.000000,3.300,...,unverified,0,0.014590,None,1,0.996847,0.0,0.0,0.302444,1711861114
2,10335832,10335832,1198787252,NaN,0.0,890,amd64,256,42.666667,2.250,...,verified,1,0.008337,None,1,0.988346,0.0,0.0,0.401667,1711861114
3,10124251,10124251,1226518510,NaN,0.0,890,amd64,32,32.000000,3.500,...,verified,1,0.010211,None,1,0.998522,0.0,0.0,0.391667,1711861114
4,10046425,10046425,1719137691,NaN,0.0,900,arm64,72,72.000000,3.456,...,verified,1,0.031388,None,1,0.992889,0.0,0.0,3.001667,1711861114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,7201173,7201173,1762364011,12837.0,0.0,890,amd64,256,128.000000,2.200,...,verified,1,0.024451,None,1,0.999058,0.0,0.0,4.401667,1711861114
60,7657164,7657164,1930589709,16742.0,0.0,890,amd64,32,32.000000,3.900,...,verified,1,0.016675,None,1,0.998754,0.0,0.0,2.001667,1711861114
61,7907891,7907891,1881186210,32023.0,0.0,800,amd64,64,10.666688,3.000,...,verified,1,0.023000,None,1,0.998664,0.0,0.0,1.501667,1711861114
62,10117059,10117059,1901320171,NaN,0.0,890,amd64,64,64.000000,2.500,...,unverified,0,0.011255,None,1,0.988595,0.0,0.0,1.201667,1711861114


In [116]:
raw.rented.astype(int).sum()

0

In [117]:
len(const.KEEP_COLS)

46

In [118]:
preprocess(raw)

In [119]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7766 entries, 0 to 7765
Data columns (total 43 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   rented               7766 non-null   int32 
 1   compute_cap          7766 non-null   int32 
 2   credit_discount_max  7766 non-null   int32 
 3   driver_version       7766 non-null   object
 4   cuda_max_good        7766 non-null   object
 5   machine_id           7766 non-null   int32 
 6   hosting_type         7766 non-null   int32 
 7   public_ipaddr        7766 non-null   object
 8   reliability          7766 non-null   int32 
 9   host_id              7766 non-null   int32 
 10  id                   7766 non-null   int32 
 11  num_gpus             7766 non-null   int32 
 12  total_flops          7766 non-null   int32 
 13  min_bid              7766 non-null   int32 
 14  dph_base             7766 non-null   int32 
 15  gpu_name             7766 non-null   object
 16  gpu_ra

In [6]:
machines_exp = get_machines()

In [8]:
machines_vast, offers = split_raw(raw)

machines_vast.set_index('machine_id', inplace=True)
machines_exp.set_index('machine_id', inplace=True)

machines_vast.sort_index(axis=1, inplace=True)
machines_exp.sort_index(inplace=True)
machines_vast.sort_index(inplace=True)

In [9]:
machines_vast

,ask_contract_id,bundle_id,bundled_results,bw_nvlink,compute_cap,cpu_arch,cpu_cores,cpu_cores_effective,cpu_ghz,cpu_name,...,start_date,static_ip,storage_cost,storage_total_cost,timestamp,total_flops,vericode,verification,vram_costperhour,webpage
machine_id,,,,,,,,,,,,,,,,,,,,,
2750,9088415,1143745147,563.0,300.000000,800,amd64,48,48.000000,2.800000,AMD EPYC 7402P 24-Core Processor,...,NaN,False,0.150,0.001667,1711860928,31.186944,1,verified,0.022000,None
5919,5516878,1349057901,20990.0,56.248001,860,amd64,128,64.000000,2.000000,None,...,NaN,True,0.150,0.001667,1711860928,59.867136,1,verified,0.008891,None
6671,7526819,1580615313,32445.0,0.000000,860,amd64,50,20.000000,1.097937,AMD EPYC 7D12 32-Core Processor,...,NaN,False,0.165,0.001833,1711860928,41.287680,1,verified,0.008254,None
7970,7899209,1780098082,13860.0,0.000000,860,amd64,28,28.000000,4.300000,Xeon® W-2175,...,NaN,False,0.150,0.001667,1711860928,141.066240,1,verified,0.009750,None
9025,6116117,1281130616,9859.0,0.000000,860,amd64,24,3.000000,3.100000,Xeon® E5-2630 v2,...,NaN,True,0.150,0.001667,1711860928,29.933568,1,verified,0.004890,None
9033,8982469,1416403039,28624.0,0.000000,860,amd64,24,6.000000,3.100000,Xeon® E5-2630 v2,...,NaN,True,0.150,0.001667,1711860928,59.867136,1,verified,0.002667,None
10178,8180888,1612432077,20892.0,0.000000,860,amd64,12,12.000000,4.300000,Core™ i5-10400F,...,NaN,True,0.150,0.001667,1711860928,62.952960,0,unverified,0.005775,None
10283,9969082,1174844258,NaN,0.000000,860,amd64,32,8.000000,3.300000,Xeon® E5-2670 0,...,NaN,True,0.150,0.001667,1711860928,20.643840,1,verified,0.005628,None
10824,10290720,1645234715,NaN,56.248001,860,amd64,16,16.000000,3.200000,Xeon® E5-2630 v3,...,NaN,False,1.000,0.011111,1711860928,137.625600,0,unverified,0.009167,None


In [123]:
machines_exp

,bw_nvlink,compute_cap,cpu_cores,cpu_name,cpu_ram,credit_discount_max,cuda_max_good,direct_port_count,disk_bw,disk_name,...,pcie_bw,public_ipaddr,reliability,score,static_ip,storage_cost,total_flops,verification,timestamp,country
machine_id,,,,,,,,,,,,,,,,,,,,,
63,0,750,24,Xeon® E5-2630 0,354,400,12.2,1000,1748,Samsung,...,110,23.245.210.159,9982,23,0,150,15,0,1697177746,US
162,0,750,32,AMD Ryzen Threadripper 1950X 16-Core Processor,63,400,11.7,1000,1767,SATA,...,125,23.245.210.159,9979,48,1,150,61,1,1697177746,US
274,0,610,32,Xeon® E5-2650 v2,63,100,12.0,1000,1531,PNY,...,58,194.44.114.10,9960,53,1,150,49,1,1697177746,UA
275,0,610,32,Xeon® E5-2650 v2,63,100,12.0,1000,1604,PNY,...,58,194.44.114.10,9961,49,1,150,49,2,1697177746,UA
283,0,610,40,Xeon® E5-2658 v2,63,100,12.0,1000,212,SSDPR,...,57,194.44.114.10,9892,50,1,150,37,1,1697177746,UA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14017,0,860,28,Xeon® E5-2680 v4,31,400,12.2,200,371,OVATION,...,115,36.72.78.0,6093,24,1,150,40,0,1697177746,ID
14018,0,890,128,AMD EPYC 7V12 64-Core Processor,504,400,12.2,1000,1360,nvme,...,127,37.136.155.159,6101,7,1,150,570,0,1697177746,FI
14019,0,860,16,Core™ i9-9900K,31,400,12.0,0,3943,SPCC,...,3,37.171.195.43,6150,2,1,150,119,0,1697177746,FR


In [124]:
mask_id = list(set(machines_vast.index).intersection(set(machines_exp.index)))
for col in machines_vast:    
    mask = machines_vast.loc[mask_id, col] != machines_exp.loc[mask_id, col]
    if mask.any():
        print('=' * 80)
        print(col)
        print(machines_vast.loc[mask_id, col][mask])
        print(machines_exp.loc[mask_id, col][mask])

disk_space
machine_id
10577    5620
12893     800
4840      630
13977     740
13992    5750
Name: disk_space, dtype: int32
machine_id
10577    5560
12893     790
4840      610
13977     720
13992    5730
Name: disk_space, dtype: int32
dlperf
machine_id
8197     63
8218    245
8249     66
8274     61
8296    146
       ... 
7997     32
8024     92
8049     34
8120    198
8135     96
Name: dlperf, Length: 677, dtype: int32
machine_id
8197     62
8218    295
8249     65
8274     65
8296    147
       ... 
7997     49
8024     94
8049     45
8120    270
8135     97
Name: dlperf, Length: 677, dtype: int32
gpu_mem_bw
machine_id
8350      377
8923      765
9090      597
9112      764
9340     3569
9892     3660
10017     759
10334     769
10374     379
10376     622
10383     766
10474     522
10492    1007
10577    3587
10865    3619
10928    3601
11218    3518
11775    3395
11808    3507
12156     502
12206     769
12436     751
12505     454
12657     511
12836     244
12941     321
4869  

In [107]:
get_machines().query('machine_id == 274')

,bw_nvlink,compute_cap,cpu_cores,cpu_name,cpu_ram,credit_discount_max,cuda_max_good,direct_port_count,disk_bw,disk_name,...,pcie_bw,public_ipaddr,reliability,score,static_ip,storage_cost,total_flops,verification,timestamp,country
1802,0,610,32,Xeon® E5-2650 v2,63,100,12.0,1000,1550,PNY,...,58,194.44.114.10,9960,53,1,150,49,1,1697173064,UA


In [92]:
get_offers().query('machine_id == 274')

,bw_nvlink,compute_cap,cpu_cores,cpu_name,cpu_ram,credit_discount_max,cuda_max_good,direct_port_count,disk_bw,disk_name,...,public_ipaddr,reliability,rented,score,static_ip,storage_cost,total_flops,verification,timestamp,country
1879,0,860,64,Xeon® E5-2698 v3,126,400,12.0,20,3219,nvme,...,184.67.78.114,8491,1,52,1,140,35,1,1697172957,CA
1880,0,860,64,Xeon® E5-2698 v3,126,400,12.0,20,3219,nvme,...,184.67.78.114,8491,0,49,1,140,35,1,1697172957,CA
1881,0,860,64,Xeon® E5-2698 v3,126,400,12.0,20,3219,nvme,...,184.67.78.114,8491,1,51,1,140,35,1,1697172957,CA
1882,0,860,64,Xeon® E5-2698 v3,126,400,12.0,20,3219,nvme,...,184.67.78.114,8491,1,50,1,140,35,1,1697172957,CA
1883,0,860,64,Xeon® E5-2698 v3,126,400,12.0,20,3219,nvme,...,184.67.78.114,8491,1,48,1,140,35,1,1697172957,CA


In [90]:
offers.query('machine_id == 10332')

,rented,compute_cap,credit_discount_max,driver_version,cuda_max_good,machine_id,hosting_type,public_ipaddr,reliability,host_id,...,end_date,storage_cost,inet_up_cost,inet_down_cost,verification,static_ip,score,timestamp,country,num_gpus_rented
5293,1,860,400,525.105.17,12.0,10332,0,184.67.78.114,8491,54909,...,1701475200,140,1,1,1,1,50,1697172780,CA,1
5294,0,860,400,525.105.17,12.0,10332,0,184.67.78.114,8491,54909,...,1701475200,140,1,1,1,1,53,1697172780,CA,0
5295,1,860,400,525.105.17,12.0,10332,0,184.67.78.114,8491,54909,...,1701475200,140,1,1,1,1,50,1697172780,CA,1
5296,1,860,400,525.105.17,12.0,10332,0,184.67.78.114,8491,54909,...,1701475200,140,1,1,1,1,52,1697172780,CA,1
5297,1,860,400,525.105.17,12.0,10332,0,184.67.78.114,8491,54909,...,1701475200,140,1,1,1,1,49,1697172780,CA,1


In [85]:
raw.query('machine_id == 10332').num_gpus

5293    1
5294    1
5295    1
5296    1
5297    1
3354    2
3355    2
1384    4
658     5
Name: num_gpus, dtype: int32

In [41]:
machines_exp[~machines_exp.index.isin(machines_vast.index)]

,bw_nvlink,client_run_time,compute_cap,cpu_cores,cpu_name,cpu_ram,credit_discount_max,cuda_max_good,direct_port_count,disk_bw,...,public_ipaddr,reliability,score,static_ip,storage_cost,total_flops,verification,vram_costperhour,timestamp,country
machine_id,,,,,,,,,,,,,,,,,,,,,
6400,0,1.1,860,28,Xeon® E5-2680 v4,63,400,12.0,100,632,...,202.86.182.106,9990,164,1,400,71,1,0.000007,1697168315,MO
11764,0,1.1,890,16,AMD Ryzen 7 5700X 8-Core Processor,31,400,12.0,100,4304,...,202.86.182.106,9978,155,1,200,81,1,0.000016,1697168315,MO


In [57]:
(~machines_vast.index.isin(machines_exp.index)).sum(), (~machines_exp.index.isin(machines_vast.index)).sum()

(0, 0)

In [56]:
set(machines_vast.columns).difference(set(machines_exp.columns))

set()

In [39]:
set(machines_exp.columns).difference(set(machines_vast.columns))

set()

In [24]:
r_dict.keys()

dict_keys(['offers'])

In [12]:
from src.fetch import fetch_single, _get_sources

sources = _get_sources()

In [13]:
raw = fetch_single(sources[0])

In [14]:
raw

,bundle_id,bw_nvlink,client_run_time,compute_cap,cpu_cores,cpu_cores_effective,cpu_name,cpu_ram,credit_balance,credit_discount,...,rentable,score,start_date,static_ip,storage_cost,total_flops,verification,verified,vram_costperhour,timestamp
0,326519410,0.0,1.1,890,256,256.0,AMD EPYC 7H12 64-Core Processor,515759,None,None,...,False,11.919773,NaN,True,0.15,1150.550016,verified,True,0.000026,1697170880
1,326539727,0.0,1.1,890,192,192.0,AMD EPYC 7R12 48-Core Processor,515814,None,None,...,False,63.616556,NaN,True,0.35,1145.044992,unverified,False,0.000016,1697170880
2,326355330,0.0,1.1,890,192,192.0,AMD EPYC 7R12 48-Core Processor,515866,None,None,...,False,49.435375,NaN,False,0.35,1145.044992,verified,True,0.000016,1697170880
3,326530779,0.0,1.1,890,192,192.0,AMD EPYC 7R12 48-Core Processor,1031961,None,None,...,False,52.456882,NaN,False,0.35,1145.044992,verified,True,0.000016,1697170880
4,326530779,0.0,1.1,890,192,192.0,AMD EPYC 7R12 48-Core Processor,1031961,None,None,...,False,55.897495,NaN,False,0.35,1145.044992,verified,True,0.000016,1697170880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7780,324943580,0.0,1.1,610,8,8.0,AMD Ryzen 5 1500X Quad-Core Processor,15980,None,None,...,False,33.049508,NaN,False,0.15,3.991552,unverified,False,0.000008,1697170880
7781,326592017,0.0,1.1,610,12,6.0,Core™ i5-10400F,15836,None,None,...,True,17.062322,1.697171e+09,False,0.15,3.991552,verified,True,0.000008,1697170880
7782,326040091,0.0,1.1,750,8,8.0,Core™ i7 950,11955,None,None,...,False,95.256475,NaN,True,0.05,3.571584,unverified,False,0.000003,1697170880
7783,326578086,0.0,1.1,750,12,12.0,AMD Ryzen 5 5600X 6-Core Processor,15864,None,None,...,True,22.412695,1.697171e+09,False,0.15,3.521280,unverified,False,0.000010,1697170880


In [15]:
fetch_single(sources[1])

,is_bid,inet_up_billed,inet_down_billed,external,webpage,logo,rentable,compute_cap,credit_balance,credit_discount,...,storage_total_cost,verification,static_ip,score,discount_rate,discounted_hourly,discounted_dph_total,rented,pending_count,timestamp
0,False,None,None,False,None,/static/logos/vastai_small2.png,False,890,None,None,...,0.0,verified,True,12.129738,0.0,0.0,9.10,False,0,1697170959
1,False,None,None,False,None,/static/logos/vastai_small2.png,False,890,None,None,...,0.0,unverified,True,62.265000,0.0,0.0,5.60,False,0,1697170959
2,False,None,None,False,None,/static/logos/vastai_small2.png,False,890,None,None,...,0.0,verified,False,48.433542,0.0,0.0,5.60,False,0,1697170959
3,False,None,None,False,None,/static/logos/vastai_small2.png,False,890,None,None,...,0.0,verified,False,52.625774,0.0,0.0,5.60,False,0,1697170959
4,False,None,None,False,None,/static/logos/vastai_small2.png,False,890,None,None,...,0.0,verified,False,55.939620,0.0,0.0,5.60,False,0,1697170959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7780,False,None,None,False,None,/static/logos/vastai_small2.png,False,610,None,None,...,0.0,unverified,False,33.988478,0.0,0.0,0.05,False,0,1697170959
7781,False,None,None,False,None,/static/logos/vastai_small2.png,True,610,None,None,...,0.0,verified,False,16.046945,0.0,0.0,0.05,False,0,1697170959
7782,False,None,None,False,None,/static/logos/vastai_small2.png,False,750,None,None,...,0.0,unverified,True,96.756329,0.0,0.0,0.02,False,0,1697170959
7783,False,None,None,False,None,/static/logos/vastai_small2.png,True,750,None,None,...,0.0,unverified,False,22.895751,0.0,0.0,0.06,False,0,1697170959


In [ ]:
r.text

In [4]:
# !python ../vast search offers "rentable = any verified = any" --raw
# !python ../vast search offers "machine_id = 9901 rentable = any verified = any" --raw -i
!python ../vast search offers "machine_id = 3056 rentable = any verified = any" --raw

https://console.vast.ai/api/v0/bundles?q=%7B%22external%22%3A+%7B%22eq%22%3A+false%7D%2C+%22machine_id%22%3A+%7B%22eq%22%3A+%223056%22%7D%2C+%22order%22%3A+%5B%5B%22score%22%2C+%22desc%22%5D%5D%2C+%22type%22%3A+%22on-demand%22%7D&api_key=6bbc6e308dcf1fb7f0ead78eae9dbfc23767871bf2b8b7e1f3c9b54a99bc1579
{'external': {'eq': False}, 'machine_id': {'eq': '3056'}, 'order': [['score', 'desc']], 'type': 'on-demand'}
[
 {
  "bundle_id": 327389754,
  "bundled_results": 2,
  "bw_nvlink": 0.0,
  "client_run_time": 1.1,
  "compute_cap": 860,
  "cpu_cores": 32,
  "cpu_cores_effective": 16.0,
  "cpu_name": "AMD EPYC 7302P 16-Core Processor",
  "cpu_ram": 257611,
  "credit_balance": null,
  "credit_discount": null,
  "credit_discount_max": 0.4,
  "cuda_max_good": 12.0,
  "direct_port_count": 50,
  "discount_rate": 0.0,
  "discounted_dph_total": 0.964,
  "discounted_hourly": 0.0,
  "disk_bw": 1800.0,
  "disk_name": "nvme",
  "disk_space": 673.5,
  "dlperf": 56.048816,
  "dlperf_per_dphtotal": 58.141925

In [8]:
# !python ../vast search offers "machine_id = 10332 rentable = any verified = any" -d
!python ../vast search offers "machine_id = 3056 verified = any" -d

https://console.vast.ai/api/v0/bundles?q=%7B%22external%22%3A+%7B%22eq%22%3A+false%7D%2C+%22machine_id%22%3A+%7B%22eq%22%3A+%223056%22%7D%2C+%22order%22%3A+%5B%5B%22score%22%2C+%22desc%22%5D%5D%2C+%22type%22%3A+%22on-demand%22%7D&api_key=6bbc6e308dcf1fb7f0ead78eae9dbfc23767871bf2b8b7e1f3c9b54a99bc1579
{'external': {'eq': False}, 'machine_id': {'eq': '3056'}, 'order': [['score', 'desc']], 'type': 'on-demand'}
ID       CUDA   N  Model     PCIE  vCPUs    RAM  Disk  $/hr    DLP    DLP/$  NV Driver   Net_up  Net_down  R     Max_Days  mach_id  status    ports  country     
6754768  12.0  2x  RTX_3090  23.5  16.0   257.6  674   0.9640  56.0   58.14  525.125.06  2748.6  2758.1    99.9  31.0      3056     verified  50     Harjumaa,_EE
6754764  12.0  1x  RTX_3090  23.3  8.0    257.6  337   0.4820  28.0   58.14  525.125.06  2748.6  2758.1    99.9  31.0      3056     verified  25     Harjumaa,_EE
6754767  12.0  4x  RTX_3090  22.9  32.0   257.6  1347  1.9280  112.1  58.14  525.125.06  2748.6  2758.

In [10]:
# !python vast search offers "rentable = any verified = any" --raw -d
# !python vast search offers "machine_id = 11833 rentable = any verified = any" --raw -d
# !python ../vast search offers "machine_id = 5919 rentable = any verified = any" --raw -d
# !python ../vast search offers "machine_id = 5919 rentable = any verified = any" -d --disable-bundling
!python ../vast search offers "machine_id = 3056 rentable = any verified = any" -d --disable-bundling

https://console.vast.ai/api/v0/bundles?q=%7B%22external%22%3A+%7B%22eq%22%3A+false%7D%2C+%22machine_id%22%3A+%7B%22eq%22%3A+%223056%22%7D%2C+%22order%22%3A+%5B%5B%22score%22%2C+%22desc%22%5D%5D%2C+%22type%22%3A+%22on-demand%22%2C+%22disable_bundling%22%3A+true%7D&api_key=6bbc6e308dcf1fb7f0ead78eae9dbfc23767871bf2b8b7e1f3c9b54a99bc1579
{'external': {'eq': False}, 'machine_id': {'eq': '3056'}, 'order': [['score', 'desc']], 'type': 'on-demand', 'disable_bundling': True}
ID       CUDA   N  Model     PCIE  vCPUs    RAM  Disk  $/hr    DLP    DLP/$  NV Driver   Net_up  Net_down  R     Max_Days  mach_id  status    ports  country     
6754764  12.0  1x  RTX_3090  23.3  8.0    257.6  337   0.4820  28.0   58.14  525.125.06  2748.6  2758.1    99.9  31.0      3056     verified  25     Harjumaa,_EE
6754768  12.0  2x  RTX_3090  23.5  16.0   257.6  674   0.9640  56.0   58.14  525.125.06  2748.6  2758.1    99.9  31.0      3056     verified  50     Harjumaa,_EE
6754769  12.0  1x  RTX_3090  23.2  8.0    

In [4]:
# query = {"external": {"eq": False},  "disable_bundling" : {"eq": True}, "type": "bid"}
query = {"external": {"eq": False},  "disable_bundling" : {"eq": True}, "type": "on-demand"}
# query = {"external": {"eq": False}, "type": "on-demand"}

# query = {'external': {'eq': False}, 'machine_id': {'eq': '3690'}, 'type': 'on-demand'}
# query = {'external': {'eq': False}, 'machine_id': {'eq': '6312'}, "disable_bundling" : {"eq": True}, 'type': 'on-demand'}

url = _apiurl("/bundles", {"q": query})
url

'https://console.vast.ai/api/v0/bundles?q=%7B%22external%22%3A+%7B%22eq%22%3A+false%7D%2C+%22disable_bundling%22%3A+%7B%22eq%22%3A+true%7D%2C+%22type%22%3A+%22on-demand%22%7D'

ImportError: cannot import name 'split_raw' from partially initialized module 'src.preprocess' (most likely due to a circular import) (C:\Users\ergot\DataspellProjects\vast-stats\src\preprocess.py)

In [7]:
r.json().keys()

dict_keys(['offers'])

In [25]:
df  = pd.DataFrame(rows)
df[['machine_id', 'id', 'num_gpus', 'rentable']].sort_values('id')

,machine_id,id,num_gpus,rentable
13,6312,5381479,1,False
12,6312,5381480,1,False
11,6312,5381481,1,False
1,6312,5381482,4,False
5,6312,5381483,2,False
10,6312,5381484,1,False
4,6312,5381485,2,False
9,6312,5516883,1,False
8,6312,5516884,1,False
7,6312,5516885,1,False


In [26]:
df  = pd.DataFrame(rows)
df[['machine_id', 'id', 'num_gpus', 'rentable']]
# df.set_index(['machine_id', 'id']).sort_index()['num_gpus']

,machine_id,id,num_gpus,rentable
0,6312,5516886,4,False
1,6312,5381482,4,False
2,6312,5516889,2,False
3,6312,5516887,2,False
4,6312,5381485,2,False
5,6312,5381483,2,False
6,6312,5516888,1,False
7,6312,5516885,1,False
8,6312,5516884,1,False
9,6312,5516883,1,False


In [27]:
df = np_min_chunk(df)

In [28]:
df_na_vals(df)

is_bid                    ->          Missing values: 0 (0.00%)
inet_up_billed            ->          Missing values: 8 (100.00%)
inet_down_billed          ->          Missing values: 8 (100.00%)
external                  ->          Missing values: 0 (0.00%)
webpage                   ->          Missing values: 8 (100.00%)
logo                      ->          Missing values: 0 (0.00%)
rentable                  ->          Missing values: 0 (0.00%)
compute_cap               ->          Missing values: 0 (0.00%)
credit_balance            ->          Missing values: 8 (100.00%)
credit_discount           ->          Missing values: 8 (100.00%)
credit_discount_max       ->          Missing values: 0 (0.00%)
driver_version            ->          Missing values: 0 (0.00%)
cuda_max_good             ->          Missing values: 0 (0.00%)
machine_id                ->          Missing values: 0 (0.00%)
hosting_type              ->          Missing values: 8 (100.00%)
public_ipaddr             ->

['inet_up_billed',
 'inet_down_billed',
 'webpage',
 'credit_balance',
 'credit_discount',
 'hosting_type',
 'cpu_name',
 'start_date',
 'end_date',
 'duration']

In [29]:
# df[df.machine_id == 10298]
df[df.machine_id == 11734][['start_date', 'end_date']]

,start_date,end_date


In [30]:
!python vast search offers "machine_id = 162 rentable = any verified = any" -d

https://console.vast.ai/api/v0/bundles?q=%7B%22external%22%3A+%7B%22eq%22%3A+false%7D%2C+%22machine_id%22%3A+%7B%22eq%22%3A+%22162%22%7D%2C+%22order%22%3A+%5B%5B%22score%22%2C+%22desc%22%5D%5D%2C+%22type%22%3A+%22on-demand%22%7D&api_key=6bbc6e308dcf1fb7f0ead78eae9dbfc23767871bf2b8b7e1f3c9b54a99bc1579
{'external': {'eq': False}, 'machine_id': {'eq': '162'}, 'order': [['score', 'desc']], 'type': 'on-demand'}
ID       CUDA   N  Model        PCIE  vCPUs   RAM  Disk  $/hr    DLP   DLP/$  NV Driver  Net_up  Net_down  R     Max_Days  mach_id  status    ports  country 
6612968  11.7  4x  RTX_2080_Ti  12.5  32.0   64.4  768   0.6000  49.3  82.08  515.65.01  45.4    345.4     99.8  -         162      verified  1000   Ohio,_US


In [31]:
df.is_bid.any()

False

In [10]:
from src.fetch import fetch_from_source, _get_sources
from src import const

sources = _get_sources()
sources[1]

{'name': 'vast.ai',
 'url': 'https://console.vast.ai/api/v0/bundles?q=%7B%22external%22%3A+%7B%22eq%22%3A+false%7D%2C+%22disable_bundling%22%3A+%7B%22eq%22%3A+true%7D%2C+%22type%22%3A+%22on-demand%22%7D',
 'timeout': 25,
 'to_split': True}

In [38]:
mach_1 = fetch_from_source(sources[0]).set_index('machine_id').sort_index()
mach_2 = fetch_from_source(sources[1]).set_index('machine_id').sort_index()

In [39]:
for col in mach_1:
    mask = mach_1[col] != mach_2[col] 
    if mask.any():
        print(mach_1[mask][col])
        print(mach_2[mask][col])
        print('=' * 80)

machine_id
283       210
1583      222
2217     2396
2501      387
3467      368
         ... 
14046    2916
14049    1208
14056    3620
14065    5795
14072    8007
Name: disk_bw, Length: 267, dtype: int32
machine_id
283        47
1583      159
2217     2573
2501      388
3467      367
         ... 
14046    3361
14049    1293
14056    4500
14065    6284
14072    4418
Name: disk_bw, Length: 267, dtype: int32
machine_id
8633      630
9146     3030
10468      70
11208    3860
Name: disk_space, dtype: int32
machine_id
8633      660
9146     3020
10468      50
11208    3850
Name: disk_space, dtype: int32
machine_id
301       62
700      111
838      228
1488     113
1520     402
        ... 
14046     23
14048     50
14050    154
14054    147
14065     69
Name: dlperf, Length: 734, dtype: int32
machine_id
301       58
700       89
838      225
1488     111
1520     341
        ... 
14046     24
14048     48
14050    172
14054    136
14065     73
Name: dlperf, Length: 734, dtype: int32
mach